In [1]:
import numpy as np

In [2]:
corpus = np.load('corpus.npy')

In [19]:
k = 10
objective = list()
centers = corpus[np.random.choice(np.arange(corpus.shape[0]), k, replace=False)]
groups = np.array([np.argmin([np.linalg.norm(center - x) for center in centers]) for x in corpus])
objective.append(
    np.array([np.linalg.norm(corpus[i] - centers[groups[i]]) for i in range(corpus.shape[0])]).sum()
)

i = 0
while True:
    centers = np.array([corpus[groups == g].mean(axis=0) for g in range(k)])
    groups = np.array([np.argmin([np.linalg.norm(center - x) for center in centers]) for x in corpus])
    objective.append(
        np.array([np.linalg.norm(corpus[i] - centers[groups[i]]) for i in range(corpus.shape[0])]).mean()
    )
    if i > 2 and ((objective[-1] - objective[-2]) + (objective[-2] - objective[-3])) < 1.0:
        break
    i += 1

In [20]:
objective[-1]

np.float64(32.814230598181794)

In [21]:
corpus[groups == 0][0]

array([0, 0, 0, ..., 0, 0, 0])

How many articles in each cluster

In [22]:
{g: corpus[groups == g].shape[0] for g in range(10)}

{0: 127, 1: 58, 2: 89, 3: 439, 4: 230, 5: 190, 6: 839, 7: 156, 8: 823, 9: 53}

Identify the words present in each cluster

In [23]:
words = np.load('words.npy')

In [24]:
cluster_word_histogram = corpus[groups == 5].sum(axis=0)

In [25]:
non_zero_i = np.where(cluster_word_histogram > 0)

In [26]:
cluster_hist_norm = cluster_word_histogram / np.linalg.norm(cluster_word_histogram)

In [27]:
cluster_hist_norm.dtype

dtype('float64')

In [28]:
print(*words[non_zero_i])

missile brother attack state child strike president picture support administration war volodymyr friday dust region sense result middle city zelensky defense boy severity store russia day federation body rubble prosecutor kharkiv cnn address minister blue debris ministry village border chief thursday lady morning evening term analyst month meanwhile kyiv germany intelligence start battle territory count figure front monitoring week invasion lot information fleet action vladimir round source fight blast design force amount storage group ammunition reserve half chance equipment replacement way armor toll tank evidence britain infantry ukraine technology moscow website paper focus enforcement court street food law affiliate return shooting course worker goal david name france ceremony paris madrid building responsibility maintenance total crowd part phone northeast time cover area list woman world ticket turkey afternoon operation order flight terror face warrant security publication sign

In [29]:
values_i = np.argsort(cluster_hist_norm)[::-1]

In [31]:
words[values_i]

['al' 'la' 'el' ... 'jorge' 'kelce' 'giorgia']
